This document is to clean the cwur df for analysis. 

Import libraries and df

In [1]:
import pandas as pd
import numpy as np
import os
import pycountry_convert as pc

In [2]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [3]:
path = r'C:\Users\justi\University Scores'

In [4]:
df = pd.read_csv(os.path.join(path, 'Original Data', 'World University Rankings', 'cwurData.csv'), index_col = False)

In [5]:
df.head()

,world_rank,institution,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year
0,1,Harvard University,USA,1,7,9,1,1,1,1,NaN,5,100.00,2012
1,2,Massachusetts Institute of Technology,USA,2,9,17,3,12,4,4,NaN,1,91.67,2012
2,3,Stanford University,USA,3,17,11,5,4,2,2,NaN,15,89.50,2012
3,4,University of Cambridge,United Kingdom,1,10,24,4,16,16,11,NaN,50,86.17,2012
4,5,California Institute of Technology,USA,4,2,29,7,37,22,22,NaN,18,85.21,2012


In [6]:
df.columns

Index(['world_rank', 'institution', 'country', 'national_rank',
       'quality_of_education', 'alumni_employment', 'quality_of_faculty',
       'publications', 'influence', 'citations', 'broad_impact', 'patents',
       'score', 'year'],
      dtype='object')

Rename columns appropriately to correlate with main data set

In [7]:
df.rename(columns = {'world_rank' : 'cwur_world_rank'}, inplace = True)
df.rename(columns = {'score' : 'cwur_score'}, inplace = True)
df.rename(columns = {'national_rank' : 'cwur_national_rank'}, inplace = True)

Remove Duplicates

In [8]:
dfsubset = df[df.duplicated()]
dfsubset

,cwur_world_rank,institution,country,cwur_national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,cwur_score,year


Identifying and making data types uniform

In [9]:
df.info() #all values seem to be the correct format. broad impact has 200 less values which are null values.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2200 entries, 0 to 2199
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   cwur_world_rank       2200 non-null   int64  
 1   institution           2200 non-null   object 
 2   country               2200 non-null   object 
 3   cwur_national_rank    2200 non-null   int64  
 4   quality_of_education  2200 non-null   int64  
 5   alumni_employment     2200 non-null   int64  
 6   quality_of_faculty    2200 non-null   int64  
 7   publications          2200 non-null   int64  
 8   influence             2200 non-null   int64  
 9   citations             2200 non-null   int64  
 10  broad_impact          2000 non-null   float64
 11  patents               2200 non-null   int64  
 12  cwur_score            2200 non-null   float64
 13  year                  2200 non-null   int64  
dtypes: float64(2), int64(10), object(2)
memory usage: 240.8+ KB


In [10]:
df.isnull().sum()

cwur_world_rank           0
institution               0
country                   0
cwur_national_rank        0
quality_of_education      0
alumni_employment         0
quality_of_faculty        0
publications              0
influence                 0
citations                 0
broad_impact            200
patents                   0
cwur_score                0
year                      0
dtype: int64

Change country names to match main document

In [11]:
df['country'].value_counts()

USA                     573
China                   167
Japan                   159
United Kingdom          144
Germany                 115
France                  109
Italy                    96
Spain                    81
Canada                   72
South Korea              72
Australia                58
Taiwan                   46
Brazil                   36
India                    31
Netherlands              29
Switzerland              26
Sweden                   24
Austria                  24
Israel                   22
Finland                  20
Turkey                   20
Belgium                  20
Poland                   18
Iran                     16
Ireland                  16
Portugal                 14
Greece                   14
Hong Kong                12
Norway                   12
New Zealand              12
Hungary                  12
Denmark                  12
South Africa             10
Czech Republic           10
Russia                    9
Saudi Arabia        

In [12]:
#change puerto rico to united states, change usa to united states of america, change taiwan to china, change hong kong to china
df['country'] = df['country'].replace('USA','United States of America', regex=True)
df['country'] = df['country'].replace('Hong Kong','China', regex=True)
df['country'] = df['country'].replace('Taiwan','China', regex=True)
df['country'] = df['country'].replace('Puerto Rico','United States of America', regex=True)

In [13]:
df['country'].value_counts()

United States of America    575
China                       225
Japan                       159
United Kingdom              144
Germany                     115
France                      109
Italy                        96
Spain                        81
Canada                       72
South Korea                  72
Australia                    58
Brazil                       36
India                        31
Netherlands                  29
Switzerland                  26
Austria                      24
Sweden                       24
Israel                       22
Belgium                      20
Turkey                       20
Finland                      20
Poland                       18
Iran                         16
Ireland                      16
Portugal                     14
Greece                       14
New Zealand                  12
Hungary                      12
Norway                       12
Denmark                      12
Czech Republic               10
South Af

Add flags

In [14]:
df['cwur_world_rank'].max()

1000

In [15]:
df.loc[df['cwur_world_rank'] <=100, 'cwur_rank_flag_top_x'] = '100'
df.loc[(df['cwur_world_rank'] > 100) & (df['cwur_world_rank'] <= 200), 'cwur_rank_flag_top_x'] = '200'
df.loc[(df['cwur_world_rank'] > 200) & (df['cwur_world_rank'] <= 300), 'cwur_rank_flag_top_x'] = '300'
df.loc[(df['cwur_world_rank'] > 300) & (df['cwur_world_rank'] <= 400), 'cwur_rank_flag_top_x'] = '400'
df.loc[df['cwur_world_rank'] > 400, 'cwur_rank_flag_top_x'] = '1000'

In [16]:
df['cwur_rank_flag_top_x'] = df['cwur_rank_flag_top_x'].astype('float64')

Create continent column

In [17]:
f = lambda x: pc.country_name_to_country_alpha2(x, cn_name_format = 'default')
df['country_code'] = df['country'].apply(f)

In [18]:
f2 = lambda x: pc.country_alpha2_to_continent_code(x)
df['continent_code'] = df['country_code'].apply(f2)

In [19]:
df['continent_code'].value_counts()

EU    826
NA    651
AS    576
OC     70
SA     57
AF     20
Name: continent_code, dtype: int64

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2200 entries, 0 to 2199
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   cwur_world_rank       2200 non-null   int64  
 1   institution           2200 non-null   object 
 2   country               2200 non-null   object 
 3   cwur_national_rank    2200 non-null   int64  
 4   quality_of_education  2200 non-null   int64  
 5   alumni_employment     2200 non-null   int64  
 6   quality_of_faculty    2200 non-null   int64  
 7   publications          2200 non-null   int64  
 8   influence             2200 non-null   int64  
 9   citations             2200 non-null   int64  
 10  broad_impact          2000 non-null   float64
 11  patents               2200 non-null   int64  
 12  cwur_score            2200 non-null   float64
 13  year                  2200 non-null   int64  
 14  cwur_rank_flag_top_x  2200 non-null   float64
 15  country_code         

In [21]:
df.describe()

,cwur_world_rank,cwur_national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,cwur_score,year,cwur_rank_flag_top_x
count,2200.000000,2200.000000,2200.000000,2200.000000,2200.000000,2200.000000,2200.000000,2200.000000,2000.000000,2200.000000,2200.000000,2200.000000,2200.000000
mean,459.590909,40.278182,275.100455,357.116818,178.888182,459.908636,459.797727,413.417273,496.699500,433.346364,47.798395,2014.318182,645.454545
std,304.320363,51.740870,121.935100,186.779252,64.050885,303.760352,303.331822,264.366549,286.919755,273.996525,7.760806,0.762130,396.353631
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,43.360000,2012.000000,100.000000
25%,175.750000,6.000000,175.750000,175.750000,175.750000,175.750000,175.750000,161.000000,250.500000,170.750000,44.460000,2014.000000,200.000000
50%,450.500000,21.000000,355.000000,450.500000,210.000000,450.500000,450.500000,406.000000,496.000000,426.000000,45.100000,2014.000000,1000.000000
75%,725.250000,49.000000,367.000000,478.000000,218.000000,725.000000,725.250000,645.000000,741.000000,714.250000,47.545000,2015.000000,1000.000000
max,1000.000000,229.000000,367.000000,567.000000,218.000000,1000.000000,991.000000,812.000000,1000.000000,871.000000,100.000000,2015.000000,1000.000000


In [22]:
df['year'].value_counts()

2014    1000
2015    1000
2012     100
2013     100
Name: year, dtype: int64

Export DF as pickle and csv

In [23]:
df.to_pickle(os.path.join(path, 'Prepared Data', 'cwur_df_cleaned.pkl'))

In [24]:
df.to_csv(os.path.join(path, 'Prepared Data', 'cwur_df_cleaned_csv.csv'))